In [1]:
from langchain_community.document_loaders import JSONLoader
from tqdm.auto import tqdm
import json
import pickle
from langchain.text_splitter import Language
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore

In [3]:
from langchain.vectorstores.weaviate import Weaviate

In [18]:
diff_loader = JSONLoader(
    file_path='../data/final_preprocessed_data/js_rag_db_data_300.json',
    jq_schema='.[].diff',
    text_content=False)

diff_data = diff_loader.load()

In [4]:
msg_loader = JSONLoader(
    file_path='../data/final_preprocessed_data/js_rag_db_data_300.json',
    jq_schema='.[].msg',
    text_content=False)

msg_data = msg_loader.load()

In [2]:
import os
client = weaviate.connect_to_wcs(
    cluster_url="https://js-sandbox-rehprp85.weaviate.network",  # Replace with your Weaviate Cloud URL
    auth_credentials=weaviate.auth.AuthApiKey("zUqPuPT0LqQH6JuGa8CkguOuomjJS8NTholY"),  # Replace with your Weaviate Cloud key
)

print(client.is_ready())

True


In [5]:
client = weaviate.Client(
    url = "https://js-sandbox-rehprp85.weaviate.network",  # Replace with your Weaviate endpoint
    auth_client_secret=weaviate.auth.AuthApiKey(api_key="zUqPuPT0LqQH6JuGa8CkguOuomjJS8NTholY"),  # Replace with your Weaviate instance API key
)

C:\Users\tyfann\anaconda3\envs\llm4commit\lib\site-packages\weaviate\warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [6]:
weaviate_client = weaviate.connect_to_local()

In [10]:
# Define the path to the pre-trained model you want to use
modelPath = "mixedbread-ai/mxbai-embed-large-v1"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda', 'trust_remote_code': True}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    cache_folder = '../models',
    
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

C:\Users\tyfann\anaconda3\envs\llm4commit\lib\site-packages\torch\__init__.py:1427: ResourceWarning: unclosed <ssl.SSLSocket fd=584, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.0.160', 11609), raddr=('34.149.137.116', 443)>
  _C._initExtension(manager_path())


In [19]:
vectorstore = Weaviate.from_texts(diff_data, embedding=embeddings, client=client, index_name="LangChain_3dc65c162ea84f5289e883328db194ba")

AttributeError: 'Document' object has no attribute 'replace'

In [31]:
db = WeaviateVectorStore.from_documents(diff_data, embeddings, client=client)

In [41]:
client.collections.get('LangChain_3dc65c162ea84f5289e883328db194ba')

In [7]:
test_diff_loader = JSONLoader(
    file_path='../data/final_preprocessed_data/js_baseline_test_data_300.json',
    jq_schema='.[].diff',
    text_content=False)

test_diff_data = test_diff_loader.load()[:1000]

with open('../data/final_preprocessed_data/js_baseline_test_data_300.json', 'r', encoding='UTF-8') as f:
    test_data = json.load(f)[:1000]

In [16]:
vectorstore.similarity_search_with_relevance_scores(test_diff_data[2].page_content)

ValueError: Error during query: [{'locations': [{'column': 6, 'line': 1}], 'message': 'Cannot query field "Text" on type "GetObjectsObj".', 'path': None}]

In [12]:
result = db.similarity_search_with_relevance_scores(test_diff_data[2].page_content)

In [16]:
retriever = db.as_retriever()
retriever.invoke(test_diff_data[2].page_content)[0]

Document(page_content='diff --git a/scripts/release/release.js b/scripts/release/release.js @@ -5,5 +5,4 @@ const git = require("simple-git")();\nlet message = "Release v" + pkg.version;\n// Commit and push to current branch\n-git.commit(message, ["dist"]).push();\n-\n+git.commit(message, ["dist", "package.json"]).push();\n', metadata={'seq_num': 10366.0, 'source': 'D:\\TU Delft\\thesis\\LLM_CMG\\llm4commit\\data\\final_preprocessed_data\\js_rag_db_data_300.json'})

In [17]:
deb = []
deb.append(None)
deb.append(1)
deb

[None, 1]

In [18]:
# db = Chroma(persist_directory="./final_js_rag_single_diff_db_300", embedding_function=embeddings)
retriever = db.as_retriever()
similar_diff = []
for index, diff_doc in tqdm(enumerate(test_diff_data), total=len(test_diff_data), desc="Processing documents"):
    # documents = splitters[1].split_documents([diff_doc])
    candidates = db.similarity_search_with_relevance_scores(diff_doc.page_content)
    if candidates[0][1]>=0.8:
        # similar_diff.append(retriever.invoke(diff_doc.page_content)[0])
        similar_diff.append(candidates[0][0])
        print(index)
    else:
        similar_diff.append(None)
    # similar_diff.append(similarity_search(documents, db))

Processing documents:   0%|          | 0/1000 [00:00<?, ?it/s]

In [29]:
msg_data[similar_diff[0].metadata['seq_num'] - 1]

TypeError: list indices must be integers or slices, not float

In [30]:

data = []

for sim_diff, test_diff in zip(similar_diff, test_data):
    item = {'org_diff': test_diff['diff'],
            'org_msg': test_diff['msg'],
            'sim_msg': msg_data[int(sim_diff.metadata['seq_num']-1)].page_content,
            # 'sim_msg': msg_data[sim_diff-1].page_content,
            # 'sim_diff': diff_data[sim_diff-1].page_content}
            'sim_diff': diff_data[int(sim_diff.metadata['seq_num']-1)].page_content}
    data.append(item)
    
# Write the data to a JSON file
with open('../data/final_preprocessed_data/js_baseline/js_baseline_weaviate_rag_300_prompt.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [19]:
test_diff_data[2]

Document(page_content="diff --git a/scripts/release/update-changelog-release-date.js b/scripts/release/update-changelog-release-date.js const fs = require('fs');\nconst cp = require('child_process');\n-const moment = require('moment');\nconst pkg = require('../../package.json');\nif (pkg.version.indexOf('-rc') !== -1) {\n", metadata={'source': 'D:\\TU Delft\\thesis\\LLM_CMG\\llm4commit\\data\\final_preprocessed_data\\js_baseline_test_data_300.json', 'seq_num': 3})

In [20]:
with open('../../../LLM4Commit/experiment_data/all_result.json', 'r', encoding='UTF-8') as f:
    llm4commit_data = json.load(f)

In [21]:
max_len = 0
for data in llm4commit_data:
    max_len = max(max_len, len(data['diff']))

In [22]:
max_len

2604

In [23]:
len(llm4commit_data)

7428

In [24]:
llm4commit_data = llm4commit_data[:100]